In [7]:
"""
!pip install --upgrade pip
!pip install simplekml
!pip install pycollada
!pip install pandas
!pip install numpy
!pip install -U ipykernel
"""

import sys
import os

os.chdir('/home/rainerth/notebooks/wind-turbine-kml/')
# os.chdir(os.path.dirname(__file__))
sys.path.append('./rthkml')

print(sys.path)

import collada_wt
import rthkml
from rthkml import *
import simplekml
# https://simplekml.readthedocs.io/en/latest/index.html

kml = simplekml.Kml()


['/home/rainerth/notebooks', '/home/rainerth/anaconda3/lib/python38.zip', '/home/rainerth/anaconda3/lib/python3.8', '/home/rainerth/anaconda3/lib/python3.8/lib-dynload', '', '/home/rainerth/.local/lib/python3.8/site-packages', '/home/rainerth/anaconda3/lib/python3.8/site-packages', '/home/rainerth/anaconda3/lib/python3.8/site-packages/locket-0.2.1-py3.8.egg', './rthkml', './rthkml']


In [8]:
# create turbine
# https://www.wind-turbine-models.com/turbines/2425-vestas-v172-7.2-enventus
# https://www.koordinaten-umrechner.de/decimal/48.237694,8.524661?karte=OpenStreetMap&zoom=16

daeurl = "models"

collada_wt1 = collada_wt.create_turbine(tower_height=199,
                    tower_bot_diameter = 6,
                    tower_top_diameter = 5,
                    nacelle_height = 3,
                    nacelle_length = 20,
                    nacelle_overhang = 8,
                    rotor_diameter = 172,
                    blade_root_length = 3.5,
                    blade_root_diameter = 3,
                    blade_chord=4,
                    blade_tip_size=0.5,
                    blade_twist=30,
                )

collada_wt1.write(daeurl + '/vestas-v172.dae')

collada_wt2 = collada_wt.create_turbine(tower_height=95,
                    tower_bot_diameter = 5,
                    tower_top_diameter = 4,
                    nacelle_height = 3,
                    nacelle_length = 10,
                    nacelle_overhang = 4,
                    rotor_diameter = 70,
                    blade_root_length = 2.5,
                    blade_root_diameter = 2,
                    blade_chord=4,
                    blade_tip_size=0.5,
                    blade_twist=30,
                )

collada_wt2.write(daeurl + '/WEA-2003.dae')


# WEA geplant
collada_zone = collada_wt.create_zone(
                    zone_height=199+172/2+100,
                    zone_diameter = 150*2+172,
                )
collada_zone.write(daeurl + '/WEA-safety-zone.dae')


# WEA 2003
collada_zone = collada_wt.create_zone(
                    zone_height=99+60/2+100,
                    zone_diameter = 150*2+60,
                )
collada_zone.write(daeurl + '/WEA-safety-zone-2003.dae')


# Testturm RW
collada_zone = collada_wt.create_zone(
                    zone_height=246,
                    zone_diameter = 24.8,
                )
collada_zone.write(daeurl + '/testturm.dae')


In [9]:
# create kml for Google Earth
# https://www.schwarzwaelder-bote.de/inhalt.windenergieanlagen-in-boesingen-drei-plus-maximal-fuenf-im-gespraech.8840b7d3-7346-4f63-8c82-27f76a697bdd.html
# read coordinates from csv

import pandas as pd
import numpy as np


def csv2df(filename):
	df = pd.read_csv(filename, sep=',', decimal='.', header=0)
	df = df.dropna(axis=0, subset=['Latitude', 'Longitude'])
	df = df[df['Latitude'] != '']
	df = df[df['Longitude'] != '']

	df['Latitude'] = df['Latitude'].astype(np.float64)
	df['Longitude'] = df['Longitude'].astype(np.float64)

	return df

def add_turbine(row, folder, folder_safety, folder_marker, markerstyle, visibility=1):
	name = row['Name']
	daemodel = row['Model']
	latitude = row['Latitude']
	longitude = row['Longitude']
	height = row['height']
	diameter = row['diameter']

	# add turbine or other model
	turbine = folder.newmodel(name=name)
	turbine.link.href = 'models/' + daemodel
	turbine.location.latitude = latitude
	turbine.location.longitude = longitude
	turbine.scale.x = 1
	turbine.scale.y = 1
	turbine.scale.z = 1
	turbine.visibility = visibility

	# add turbine as zylinder
	create_kml_cylinder (
		kmlfolder=folder,
		cyl_center=(longitude, latitude),
		cyl_diameter=diameter,
		cyl_height=height + diameter/2,
		cyl_color=simplekml.Color.red,
		cyl_name=name,
		cyl_visibility=1
	)

	# add safety zone as zylinder
	# add safety zone for turbine
	if folder_safety is not None:
		create_kml_cylinder (
			kmlfolder=folder_safety,
			cyl_center=(longitude, latitude),
			cyl_diameter=diameter + 150*2,
			cyl_height=height + diameter/2 + 100,
			cyl_color=simplekml.Color.lightgray,
			cyl_name=name,
			cyl_visibility=1
		)


	# add safety zone for turbine
	if folder_safety is not None:
		# add safety zone
		safety = folder_safety.newmodel(name=name)
		safety.link.href = 'models/WEA-safety-zone.dae'
		safety.location.latitude = latitude
		safety.location.longitude = longitude
		safety.scale.x = 1
		safety.scale.y = 1
		safety.scale.z = 1
		safety.visibility = 0

	# add marker for turbine
	if markerstyle is not None:
		location = folder_marker.newpoint(name=name, coords=[(longitude, latitude)])
		location.style=markerstyle



	return

# simple WEA locations with icon
markerstyle = simplekml.Style()
markerstyle.labelstyle.color = simplekml.Color.lightgray  # Make the text red
markerstyle.labelstyle.scale = 0.8  # Make the text twice as big
markerstyle.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/shapes/placemark_circle.png'
markerstyle.iconstyle.color = simplekml.Color.darkgray

markerstyle_badenova = simplekml.Style()
markerstyle_badenova.labelstyle.color = simplekml.Color.lightgray  # Make the text blue
markerstyle_badenova.labelstyle.scale = 0.8  # Make the text twice as big
markerstyle_badenova.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/shapes/placemark_circle.png'
markerstyle_badenova.iconstyle.color = simplekml.Color.lightblue

markerstyle_iterra = simplekml.Style()
markerstyle_iterra.labelstyle.color = simplekml.Color.lightgray  # Make the text red
markerstyle_iterra.labelstyle.scale = 0.8  # Make the text twice as big
markerstyle_iterra.iconstyle.icon.href = 'http://maps.google.com/mapfiles/kml/shapes/placemark_circle.png'
markerstyle_iterra.iconstyle.color = simplekml.Color.red

folder_marker = kml.newfolder(name='Standorte')

# Badenova
folder = kml.newfolder(name='badenova')
folder_safety = folder.newfolder(name='Safety')

df = csv2df('locations/badenova.csv')
df.apply(lambda row: add_turbine(row, folder, folder_safety, folder_marker, markerstyle_badenova, visibility=1), axis=1)


# iTerra
folder = kml.newfolder(name='iTerra')
folder_safety = folder.newfolder(name='Safety')

df = csv2df('locations/iterra.csv')
df.apply(lambda row: add_turbine(row, folder, folder_safety, folder_marker, markerstyle_iterra, visibility=1), axis=1)

# Dunningen
folder = kml.newfolder(name='Dunningen')
folder_safety = folder.newfolder(name='Safety')

df = csv2df('locations/dunningen.csv')
df.apply(lambda row: add_turbine(row, folder, folder_safety, folder_marker, markerstyle, visibility=1), axis=1)

# Herrenzimmern
folder = kml.newfolder(name='Herrenzimmern')
folder_safety = folder.newfolder(name='Safety')

df = csv2df('locations/herrenzimmern.csv')
df.apply(lambda row: add_turbine(row, folder, folder_safety, folder_marker, markerstyle, visibility=1), axis=1)

# Testturm
folder = kml.newfolder(name='Testturm')

df = csv2df('locations/testturm.csv')
df.apply(lambda row: add_turbine(row, folder, None, None, None, visibility=1), axis=1)






0    None
1    None
dtype: object

In [10]:
# add return range to kml
import rthkml

folder = kml.newfolder(name='Endanflugbereiche')

# Mittelpunkt und Radius des Kreises
airfield_boesingen = (8.534621455251553, 48.227697118500274)
radius_m = 5000	# in Koordinaten-Einheiten (Grad)
height = 500.0

create_kml_cylinder (
	kmlfolder=folder,
    cyl_center=airfield_boesingen,
    cyl_diameter=radius_m * 2,
    cyl_height=height,
    cyl_color=simplekml.Color.green,
    cyl_name='Endanflugbereich',
	cyl_visibility=0
)




In [11]:
# save models to kml

kml_file = './output/WEA-boesingen.kml'
kml.save(kml_file)

In [12]:
# create kmz from kml and dae files

import zipfile

def create_kmz(kml_filename, dae_filenames, output_filename):
    with zipfile.ZipFile(output_filename, 'w') as kmz:
        kmz.write(kml_filename)

        for dae in dae_filenames:
            kmz.write(dae, os.path.join("models", os.path.basename(dae)))

#if __name__ == "__main__":

# Listet alle .dae-Dateien im "model"-Verzeichnis auf

dae_files = [os.path.join("models", f) for f in os.listdir("models") if f.endswith('.dae')]
kml_file = "./output/WEA-boesingen.kml"  # Ersetzen Sie dies durch den Pfad zu Ihrer KML-Datei
output_file = "./output/WEA-boesingen.kmz"
create_kmz(kml_file, dae_files, output_file)


## Quellen

* https://www.landesrecht-bw.de/jportal/?quelle=jlink&docid=JURE060017391&psml=bsbawueprod.psml&max=true&doc.part=L&doc.norm=all
* https://www.daec.de/media/files/2022/Fachbereiche/Umwelt/Leitfaden_Luftfahrthindernisse_14_03_2022.pdf
* https://www.fsco.de/images/Dokumente/Studie_DLR.pdf
* https://www.dhv.de/fileadmin/user_upload/aktuell_zu_halten/Gelaende/DHV_info_199_windkraft.pdf
* http://www.pontepress.de/pdf/u12_201706.pdf
